# Защита персональных данных клиентов

Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и обосновать корректность его работы.При преобразовании качество моделей машинного обучения не ухудшилось (простая модель линейной регрессии без подбора гиперпараметров)

## Загрузка данных

Сделаем все необходимые импорты

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import random

Откроем файл и изучим его

In [3]:
insurance = pd.read_csv('/datasets/insurance.csv')
display(insurance.head(5))
print(insurance.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


Данные все числовые и не содержат пропусков. Проверим, есть ли в датасете заведомо некорректные данные (слишком маленький или отрицательный возраст, например, или количество членов семьи). Для этого выведем минимальное и максимальное значение в каждом столбце:

In [4]:
print('Минимальные значения в каждом столбце')
print(insurance.min())
print()
print('Максимальные значения в каждом столбце')
print(insurance.max())

Минимальные значения в каждом столбце
Пол                     0.0
Возраст                18.0
Зарплата             5300.0
Члены семьи             0.0
Страховые выплаты       0.0
dtype: float64

Максимальные значения в каждом столбце
Пол                      1.0
Возраст                 65.0
Зарплата             79000.0
Члены семьи              6.0
Страховые выплаты        5.0
dtype: float64


Можно видеть, что значения каждого признака находятся в допустимом диапазоне. Для удобства работы переименуем колонки

In [5]:
insurance.columns = ['gender', 'age', 'salary', 'family_members', 'payments']
print(insurance.columns)

Index(['gender', 'age', 'salary', 'family_members', 'payments'], dtype='object')


Данные готовы к дальнейшей работе

## Умножение матриц

<h3>Построение модели линейной регрессии</h3>

Напишем функцию построения модели линейной регрессии и получения предсказаний, оценки функции минимизации потерь

In [6]:
def make_linear_regression(features, target):
    model = LinearRegression()
    model.fit(features, target)
    prediction = model.predict(features)
    r2 = r2_score(target,prediction)
    return model,r2

Напишем функцию для разбиения датасета на features и target:

In [7]:
def split_data(data,column):
    target = data[column]
    features = data.drop(column,axis = 1)
    return target, features

<h3>Оценка влияния на качество линейной регрессии умножения признаков на обратимую матрицу</h3>

Выясним, изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу

Выделим отдельно обучающие и целевой признак, обучим на них модель линейной регрессии и получим признак r2:

In [8]:
ins_target, ins_features = split_data(insurance,'payments')
b_model,b_r2 = make_linear_regression(ins_features, ins_target)
print('r2 = ',b_r2)

r2 =  0.42494550286668


Зададим случайную матрицу, и умножим на нее матрицу признаков. Обучим модель и посчитаем r2. Повторим экперимент 10000 раз. Выведем минимальное, максмимальное и среднее значение r2.

In [10]:
r2s = []
for i in range(10000):
    matrix = np.random.normal(size = (4,4))
    if np.linalg.det(matrix) !=0: #если матрица обратима
        ch_features = ins_features @ matrix 
        model, r2 = make_linear_regression(ch_features, ins_target)
        r2s.append(r2)
r2s = np.array(r2s)
print('min = ',np.min(r2s))
print('max = ',np.max(r2s))
print('mean = ',np.mean(r2s))

min =  0.4249455027539569
max =  0.4249455028897814
mean =  0.42494550286667326


**Ответ:** Как можно видеть, значения r2 практически неизменны. Среднее значение r2 максимально близко к значению r2 на изначальных данных. То есть, можно сказать, что умножение признаков на обратимую матрицу не меняет качество линейной регрессии. Посмотрим, как так получилось.

**Обоснование:** Известно, что модель линейной регрессии в кратком виде записывается следующим образом:
$$a = Xw$$
Также известно, оптимальные значения вектора w для модели линейной регрессии находятся по формуле:
$$w = (X^TX)^{-1}X^Ty$$
Если матрицу признаков умножить на обратимую (а, значит, квадратную) матрицу M, то данное выражение примет следующий вид:
$$w_m = ((XM)^T(XM))^{-1}(XM)^Ty$$
Преобразуем данное выражение:<br>
Для начала воспользуемся свойством транспонированного произведения матриц:
$$w_m = (M^TX^T(XM))^{-1}M^TX^Ty$$
А теперь воспользуемся сочетательным свойством умножения матриц:
$$w_m = ((M^TX^TX)M)^{-1}M^TX^Ty$$
Заметим, что матрица $X^TX$ - квадратная матрица той же размерности, что и матрица <b>M</b>, поэтому матрица $M^TX^TX$ - также квадратная матрица, поэтому можем воспользоваться свойством обратной матрицы:
$$w_m = M^{-1}(M^TX^TX)^{-1}M^TX^Ty$$
или
$$w_m = M^{-1}(X^TX)^{-1}(M^T)^{-1}M^TX^Ty$$
Известно, что $(M^T)^{-1}M^T$ - единичная матрица (из определения обратной матрицы). Тогда:
$$w_m = M^{-1}(X^TX)^{-1}EX^Ty$$
Применив сочетательное свойство умножения матриц и свойства единичной матрицы, получаем:
$$w_m = M^{-1}(X^TX)^{-1}X^Ty$$
Соответственно
$$w_m = M^{-1}w$$
Тогда, подставив значения в формулу линейной регрессии, получим:
$$a_m = (XM)( M^{-1}w)$$
или
$$a_m = XMM^{-1}w$$
или
$$a_m = XEw$$
или
$$a_m = Xw$$
или
$$a_m = a$$

Таким образом можем видеть, что вектор предсказаний при умножении матрицы признаков на обратимую матрицу не меняется. <br>
Проверим полученное значение:
$$w_m = M^{-1}w$$

In [11]:
print('Матрица коэффициентов исходной модели')
print(b_model.coef_)
print()
print('Матрица коэффициентов модели, полученной путем умножения матрицы признаков на обратимую')
print(model.coef_)
print()
print('Обратимая матрица, на которую умножали матрицу признаков:')
print(matrix)
print()
print('Произведение обратной матрицы на матрицу коэффициентов исходной модели')
print(np.linalg.inv(matrix).dot(b_model.coef_))

Матрица коэффициентов исходной модели
[ 7.92580543e-03  3.57083050e-02 -1.70080492e-07 -1.35676623e-02]

Матрица коэффициентов модели, полученной путем умножения матрицы признаков на обратимую
[ 0.02971531  0.01893548 -0.00796993  0.0344522 ]

Обратимая матрица, на которую умножали матрицу признаков:
[[ 0.32313763  0.24485021  0.52992848 -0.06064033]
 [ 0.22743842  1.92260967  0.76936364 -0.03842593]
 [-1.31982233 -0.11124363  0.06873612  1.21539523]
 [-1.10355278  0.3287019   0.80199797  0.56288125]]

Произведение обратной матрицы на матрицу коэффициентов исходной модели
[ 0.02971531  0.01893548 -0.00796993  0.0344522 ]


<b>Вывод</b><br>
Как можно видеть, матрица коэффициентов модели, полученной пцтем умножения матрицы признаков на обратимую матрицу, равна произведению обратной матрицы на матрицу коэффициентов исходной модели. Поэтому, как можно видеть, умножение матрицы признаков на обратимую, не приводит к изменению предсказаний и, соответственно, качества модели. Это можно использовать для создания алгоритма преобразования данных.

## Алгоритм преобразования

<h3>Алгоритм</h3>

В предыдущем пункте мы выяснили, что умножение признаков на обратимую матрицу не приводит к изменению результатов линейной регрессии. Однако, если мы преобразуем только признаки, то столбец 'payments' остается неизменным. Нам необходимо внести изменения и в этот столбец тоже. <br>
Для этого, прежде чем умножать признаки на обратимую матрицу, необходимо произвести некоторые изменения в полном датасете.
<br>
<br>
Таким образом, можно предложить следующий алгоритм для системы кодирования:

1. Сгенерировать случайную обратимую матрицу, вычсилить ее определитель</li>
2. Преобразовать исходные данные (вместе с целевым признаком) следующим способом: Сначала поэлементно прибавить случайное число, потом поэлементно умножить на другое случайное число. Эти случайные числа не должны быть легко вычисляемыми. ИХ нужно сделать еще более сложными для понимания. Например, следующим способом:<br>
        2.1. Сгенерировать случайное число в заданном диапазоне, случайным образом сделав его положительным или отрицательным. Данное число умножить на определитель матрицы. Таким образом получим случайное число, которое будем поэлементно прибавлять.
        2.2. Сгенерировать случайное число в другом диапазоне, случайным образом сделав его положительным или отрицательным. Умножить его на определитель матрицы. Возвести все в куб. Таким образом получим число, на которое будем поэлементно умножать
3. Умножить признаки на сгенерированную обратимую матрицу

<br>
Проверим и обоснуем этот алгоритм.

<h3>Обоснование алгоритма</h3>

Для начала необходимо убедиться, что при поэлементном умножении или прибавлении какого-либо отличного от нуля числа исходной матрицы (признаков вместе с целевым) качество модели не меняется: 

Чтобы было проще сравнивать значения коэффициентов и r2, создадим специальный датасет для этого

In [12]:
#функция для добавления строки с коэффициентами в датасет
def add_coeff(mod,r2,name,coeff_data):
    new_row = {'_name':name,'_r2':r2,'_w1':mod.coef_[0],'_w2':mod.coef_[1],'_w3':mod.coef_[2],'_w4':mod.coef_[3],'w0 - free':mod.intercept_}
    coeff_data = coeff_data.append(new_row,ignore_index = True)
    display(coeff_data)
    return coeff_data

In [13]:
coeff_data = pd.DataFrame()
coeff_data = add_coeff(b_model,b_r2,'модель на исходных данных',coeff_data)

,_name,_r2,_w1,_w2,_w3,_w4,w0 - free
0,модель на исходных данных,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,-0.938236


Напишем функцию обработки значения датасета, построения модели линейной регрессии на новых данных и вывода значений коэффициентов:

In [14]:
def find_characteristics(data, target_column, model_name,coeff_data):
    print('Первые 3 строки измененного датасета')
    display(data.head(3))
    ins_ch_target,ins_ch_features = split_data(data,'payments')
    b2_model,b2_r2 = make_linear_regression(ins_ch_features, ins_ch_target)
    print('Значения коэффициентов:')
    coeff_data = add_coeff(b2_model,b2_r2,model_name,coeff_data)
    return coeff_data

**Убедимся, что, если мы поэлементно прибавим какое-то число, то значение r2 у нас не изменится:**

In [15]:
#в нашем примере прибавим число 3:
ins_chd = pd.DataFrame((np.array(insurance)+3),columns = insurance.columns, index = insurance.index)

coeff_data = find_characteristics(ins_chd, 'payment', 'модель на данных плюс число 3 поэлементно',coeff_data)

Первые 3 строки измененного датасета


,gender,age,salary,family_members,payments
0,4.0,44.0,49603.0,4.0,3.0
1,3.0,49.0,38003.0,4.0,4.0
2,3.0,32.0,21003.0,3.0,3.0


Значения коэффициентов:


,_name,_r2,_w1,_w2,_w3,_w4,w0 - free
0,модель на исходных данных,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,-0.938236
1,модель на данных плюс число 3 поэлементно,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,1.971566


Можно видеть, что если к исходному датасету поэлементно прибавить некоторое число, то значение r2 не меняется.Также остаются неизменными коэффициенты w, меняется только w0, влияющий непосредственно на сдвиг (что естественно: все значения в датасете как бы сдивнулись по оси Y).Посмотрим, как это получилось: <br>
При прибавлении некоторого числа k к элементу $x_{ij}$ получаем новое значение: $x_{ij}+k$. Таким образом, для i-ой строки получаем:
$$ w_1*(x_{i1}+k) + w_2*(x_{i2}+k) +...+w_n*(x_{in}+k) + w_0 = y_i+k $$
или
$$ w_1*x_{i1}+w_1*k + w_2*x_{i2}+w_2*k +...+w_n*x_{in}+w_n*k +w_0 = y_i+k $$
или 
$$ w_1*x_{i1} + w_2*x_{i2} +...+w_n*x_{in} + \underbrace{(w_0+w_1*k+w_2*k+...+w_n*k)}_{изменение} = y_i+k $$

Можно видеть, что при прибавлении к эпоэлементно какого-либо числа в модели изменится только w0. Таким образом, получаем, что прибавление произвольного числа ко всем элементам матрицы приведет к изменению коэффициента w0 в модели линейной регрессии. Все остальные коэффициенты остаются неизменными, как и r2

**Убедимся, что, если мы умножим исходные данные на какое-то число поэлементно, то r2 от этого не изменится:**

In [16]:
#в нашем примере умножим число 5:
ins_chd = pd.DataFrame((np.array(insurance)*5),columns = insurance.columns, index = insurance.index)
coeff_data = find_characteristics(ins_chd, 'payment', 'модель на данных умноженных на 5 поэлементно',coeff_data)

Первые 3 строки измененного датасета


,gender,age,salary,family_members,payments
0,5.0,205.0,248000.0,5.0,0.0
1,0.0,230.0,190000.0,5.0,5.0
2,0.0,145.0,105000.0,0.0,0.0


Значения коэффициентов:


,_name,_r2,_w1,_w2,_w3,_w4,w0 - free
0,модель на исходных данных,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,-0.938236
1,модель на данных плюс число 3 поэлементно,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,1.971566
2,модель на данных умноженных на 5 поэлементно,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,-4.691178


Можем видеть, что при умножении исходного датасета на число поэлементно, значения коэффициентов W и r2 остались неизменны,  изменился только коэффициент w0 и изменился он ровно в 5 раз (именно на это число мы умножали в примере). Действительно:
$$ w_1*x_{i1}*k + w_2*x_{i2}*k +...+w_n*x_{in}*k + w_0 = y_i*k $$
или
$$ w_1*x_{i1} + w_2*x_{i2} +...+w_n*x_{in} + \frac{w_0}{k} = y_i $$

**Обоснуем, почему был выбран такой алгоритм формирования коэффициентов**

Доя начала поймем, для чего нам 2 коэффициента: Если мы будем просто умножать на какой-то коэффициент, то восстановить по поучившимся значениям данные в колонке с целевым признаком будет очень легко: нули так и останутся нулями, а все остальные числа будут кратны множителю. Его найти не составит труда. <br>
Если же мы будем просто прибавлять число, то опять же, совершенно не составит никакого труда подобрать число, которое можно вычесть отовсюду. Поэтому мы сначала прибавляем, а потом умножаем. В данном случае подобрать числа будет гораздо сложнее, особенно, если коэффициенты дробные. 

Коэффициенты, которые мы добираем для поэлементного прибавления и умножения, не должны быть легко идентифицируемы. 
Для начала мы генерируем случайные числа. Числа генерируем из разных диапазонов, чтобы они больше друг от друга отличались: тогда их сложнее поодбрать. Далее. Чтобы было еще более непонятно, случайным образом делаем эти числа отрицательными или положительными.
Также всем известно, что случайные числа генерируются генератором псевдослучайных чисел. Любое псевдослучайное число не может быть защищено от взлома (всегда найдется тот, кто знает, как работает этот генератор). Поэтому я решила усложнить немного создание этих коэффициентов:
1. коэффициент k1 умножить на еще одно псевдослучайное вещественное число. например, на определитель матрицы случайных чисел, которую мы сгенерировали ранее.
2. коэффициент k2 также умножить на этот определитель и полученное значение возвести в куб (чтобы сохранить знак)

**Обоснуем, для чего нам необходимо умножать признаки на случайную обратимую матрицу**

При использовании алгориотма "просто прибавить число и все умножить на другое число" на всем датасете остается проблема: одинаковые значения будут кодироваться одинаковым образом. Например, в поле gender всего 2 значения: 0 и 1, то есть и в закодированном виде их будет только 2, хоть и выглядеть они будут иначе. Уже зная это, можно восстановить большую часть информации. Поэтому, чтобы этого избежать, умножим признаки на обратимую матрицу. Так даже изначально одинаковые значения будут выглядеть по-разному. А когда необходимо будет раскодировать, просто умножим на обратную матрицу. 

При умножении признаков вместе с целевым признаком на обратимую матрицу столбец целевого признака меняется непропорционально - значение в каждой строке будет меняться совершенно по-разному, в этом случае мы получаем совсем другую матрицу, для которой будет строиться совершенно другая модель, поэтому и значение r2 будет другим. Поэтому мы не можем умножать на обратимую матрицу все признаки вместе с целевым

Реализуем и проверим алгоритм

## Проверка алгоритма

Напишем функцию для получения случайной матрицы и ее определителя

In [17]:
def get_matrix_and_det(size_xy):
    matrix = np.random.normal(size = (size_xy,size_xy))
    if np.linalg.det(matrix)!=0:
        return matrix, np.linalg.det(matrix)
    else:
        get_matrix_and_det(size_xy)

Напишем функцию для шифрования данных:

In [21]:
def encode_data(data, target_column):
    size_xy = data.shape[1] - 1
    matrix, det = get_matrix_and_det(size_xy)
    k1 = (-1)**random.randint(1,2)*(random.uniform(50,300))
    k2 = (-1)**random.randint(1,2)*(random.uniform(0.001,70))
    data_new = (data + k1*det) * (k2*det)**3    
    data_target, data_features = split_data(data_new,target_column)
    data_encoded = pd.DataFrame(np.array(data_features).dot(matrix),columns = data_features.columns,index = data_features.index)
    data_encoded[target_column] = data_target
    return data_encoded, matrix, k1, k2

Закодируем данные и проверим результат

In [22]:
ins_encoded,matrix,k1,k2 = encode_data(insurance,'payments')
coeff_data = find_characteristics(ins_encoded, 'payment', 'модель на зашифрованных данных',coeff_data)

Первые 3 строки измененного датасета


,gender,age,salary,family_members,payments
0,-2.308882e+11,-2.493306e+11,5.325423e+11,-3.797792e+11,-3.414315e+09
1,-1.784086e+11,-1.935413e+11,4.067009e+11,-2.896218e+11,-3.408877e+09
2,-1.016269e+11,-1.118908e+11,2.220568e+11,-1.574742e+11,-3.414315e+09


Значения коэффициентов:


,_name,_r2,_w1,_w2,_w3,_w4,w0 - free
0,модель на исходных данных,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,-9.382355e-01
1,модель на данных плюс число 3 поэлементно,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,1.971566e+00
2,модель на данных умноженных на 5 поэлементно,0.424946,0.007926,0.035708,-1.700805e-07,-0.013568,-4.691178e+00
3,модель на зашифрованных данных,0.424946,-0.083997,0.070748,3.981049e-02,0.060693,-3.316761e+09


<b>Вывод</b><br>
Можем видеть, что у модели на зашифрованных данных значение r2 совпадает со всеми предыдущими значениями этой функции минимизации потерь. Все коэффициенты отличаются, но они и должны отличаться, как мы это выяснили ранее.
<br>
Алгоритм может быть применим для шифрования без потери качества модели линейной регрессии